In [1]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor

In [2]:
class conv_block(nn.Module):
  def __init__(self, in_c, out_c):
    super(conv_block, self).__init__()
    self.conv1=nn.Conv2d(in_c, out_c, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1=nn.BatchNorm2d(out_c)
    self.conv2=nn.Conv2d(out_c, out_c, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2=nn.BatchNorm2d(out_c)
    self.relu = nn.ReLU()

  def forward(self, inputs):
    x=self.conv1(inputs)
    x=self.bn1(x)
    x=self.relu(x)
    x=self.conv2(x)
    x=self.bn2(x)
    x=self.relu(x)
    return x

In [3]:
class encoder_block(nn.Module):
  def __init__(self, in_c, out_c):
    super(encoder_block, self).__init__()
    self.conv=conv_block(in_c, out_c)
    self.pool=nn.MaxPool2d(kernel_size=2, stride=2)

  def forward(self, inputs):
    x=self.conv(inputs)
    p=self.pool(x)
    return x,p

In [4]:
class decoder_block(nn.Module):
  def __init__(self, in_c, out_c):
    super(decoder_block, self).__init__()
    self.up=nn.ConvTranspose2d(in_c, out_c, kernel_size=2, stride=2, padding=0)
    self.conv=conv_block(out_c*2, out_c)

  def forward(self, inputs, skip):
    x=self.up(inputs)
    x=torch.cat([x, skip], axis=1)
    x=self.conv(x)
    return x

In [5]:
class unet(nn.Module):
  def __init__(self, in_c=3, out_c=1):
    super(unet, self).__init__()
    self.e1=encoder_block(in_c, 64)
    self.e2=encoder_block(64, 128)
    self.e3=encoder_block(128, 256)
    self.e4=encoder_block(256, 512)   
    
    self.base=conv_block(512, 1024)

    self.d1=decoder_block(1024, 512)
    self.d2=decoder_block(512, 256)
    self.d3=decoder_block(256, 128)
    self.d4=decoder_block(128, 64)
    
    self.output=nn.Conv2d(64, out_c, kernel_size=3, padding=1)

  def forward(self, inputs):
    s1, p1=self.e1(inputs)
    s2, p2=self.e2(p1)
    s3, p3=self.e3(p2)
    s4,p4=self.e4(p3)

    b=self.base(p4)

    o1=self.d1(b, s4)
    o2=self.d2(o1, s3)
    o3=self.d3(o2, s2)
    o4=self.d4(o3, s1)

    outputs=self.output(o4)

    return outputs




In [6]:
def test():
  x = torch.randn((3,1,160,160))
  model = unet(in_c = 1, out_c= 2)
  preds = model(x)
  return preds.shape
test()

torch.Size([3, 2, 160, 160])

In [7]:
from torch.utils.data import Dataset
import os 
import numpy as np
from PIL import Image
class ForestDataset(Dataset):
  def __init__(self, image_dir, mask_dir, transform=None):
    self.image_dir = image_dir
    self.mask_dir = mask_dir
    self.transform = transform
    self.images = os.listdir(image_dir)

  def __len__(self):
    return len(self.images)

  def __getitem__(self, index):
    image_path = os.path.join(self.image_dir, self.images[index])
    mask_path = os.path.join(self.mask_dir, self.images[index].replace("sat", "mask"))
    image = np.array(Image.open(image_path).convert("RGB"))
    mask = np.array(Image.open(mask_path).convert("L"), dtype = np.float32)
    image = np.transpose(image/255.0)
    mask = np.transpose(mask/255.0)

    return image, mask
  



In [8]:
train_image_dir = "/content/drive/MyDrive/ForestAreaDataset/Forest Segmented/Forest Segmented/train_images"
test_image_dir = "/content/drive/MyDrive/ForestAreaDataset/Forest Segmented/Forest Segmented/test_images"
train_mask_dir = "/content/drive/MyDrive/ForestAreaDataset/Forest Segmented/Forest Segmented/train_masks"
test_mask_dir = "/content/drive/MyDrive/ForestAreaDataset/Forest Segmented/Forest Segmented/test_masks"

In [9]:
def check_accuracy(loader, model, device=torch.device("cuda")):
  num_correct = 0
  num_pixels = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x=x.to(device).float()
      y=y.to(device).unsqueeze(1).float()
      pred = torch.sigmoid(model(x))
      pred = (pred>0.5).float()
      num_correct += (pred==y).sum()
      num_pixels += torch.numel(pred)
    print(f"Accuracy = {(num_correct/num_pixels)*100:.2f}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor

from tqdm import tqdm

device = torch.device("cuda")

num_epochs = 5
batch_size = 10
learning_rate = 0.001

train_dataset = ForestDataset(train_image_dir, train_mask_dir, transform=ToTensor())
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_dataset = ForestDataset(test_image_dir, test_mask_dir, transform=ToTensor())
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
model = unet(in_c=3, out_c=1).to(device)

loss_f = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  model.train()
  current_loss = 0.0

  for batch_idx, (inputs, targets) in enumerate(train_dataloader):
    inputs = inputs.to(device).float()
    targets = targets.to(device).unsqueeze(1).float()

    optimizer.zero_grad()

    outputs = model(inputs)
    loss = loss_f(outputs, targets)

    loss.backward()
    optimizer.step()

    current_loss = loss.item()

  check_accuracy(test_dataloader, model, device=device)
  print(current_loss)